In [55]:
import requests;
import psycopg2;
import json;

In [56]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="joostdelaere",
    user="joostdelaere",
    password=""
)

cursor = conn.cursor()

# Delete the table
delete_table_query = "DROP TABLE IF EXISTS map_data"
cursor.execute(delete_table_query)

# Create the table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS map_data (
                    key TEXT PRIMARY KEY,
                    lsu_amount DECIMAL,
                    epoch DECIMAL,
                    staking_rewards DECIMAL,
                    interest_earnings DECIMAL
                )''')

In [57]:
import requests

url = "https://rcnet.radixdlt.com/status/gateway-status"

# Make a POST request to the Gateway Status endpoint
response = requests.post(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Print the entire response for analysis
    print(data)
else:
    # Print an error message if the request was not successful
    print("Error:", response.status_code)


{'ledger_state': {'network': 'kisharnet', 'state_version': 20436283, 'proposer_round_timestamp': '2023-07-03T20:56:09.83Z', 'epoch': 11678, 'round': 115}, 'release_info': {'release_version': '0.3.1-develop', 'open_api_schema_version': '0.3.0'}}


In [58]:
url = "https://rcnet.radixdlt.com/state/entity/details"

# Prepare the request body
request_body = {
    "addresses": [
        "component_tdx_c_1qved97g6ge57lkx95fza63hg2nn8aryjwj3tg596sgxs64u5sz"
    ],
    "aggregation_level": "global"
}

# Make a POST request to the State Entity Details endpoint
response = requests.post(url, json=request_body)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
        # Extract the 'items' list from the 'data' variable
    items = data['items']

    # Extract the 'details' dictionary from each item
    details = item['details']

    # Extract the 'state' dictionary from the 'details' dictionary
    state = details['state']

    # Extract the 'data_json' dictionary from the 'state' dictionary
    data_json = state['data_json']

    # Extract the 'data_json' dictionary from the 'state' dictionary
    fields = data_json['fields']

    # Extract the 'data_json' dictionary from the 'state' dictionary
    map_entry = fields[6]

    # Access the 'entries' list from the map entry
    entries = map_entry['entries']    

    print(entries)

    # Iterate over the map entries and add them to the dictionary
    for entry in entries:
        key = entry[0]
        epoch = entry[1][0]
        lsu_amount = entry[1][1]
        staking_rewards = entry[1][2]
        interest_earnings = entry[1][3]
        
        # Insert the values into the table
        insert_query = """
        INSERT INTO map_data (key, epoch, lsu_amount, staking_rewards, interest_earnings)
        VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (key, epoch, lsu_amount, staking_rewards, interest_earnings))
    
    # Commit the changes to the database
    conn.commit()

    # Close the database connection
    conn.close()

else:
    # Print an error message if the request was not successful
    print("Error:", response.status_code)


[['#1#', ['10899', '50', '33.3333333333333333', '0']], ['#2#', ['10899', '100', '66.6666666666666666', '0']], ['#3#', ['11098', '50', '0', '0']]]
